In [ ]:
%matplotlib inline

In [ ]:
from typing import Callable
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
from scipy import constants
from scipy import integrate
from scipy import optimize

# PHYS 395 - week 4

**Matt Wiens - #301294492**

This notebook will be organized similarly to the lab script, with major headings corresponding to the headings on the lab script.

*The TA's name (Ignacio) will be shortened to "IC" whenever used.*

## Setup

In [ ]:
# Set default plot size
plt.rcParams["figure.figsize"] = (10, 7)

In [ ]:
%%javascript
IPython.OutputArea.auto_scroll_threshold = 9999

# Root finding

Before looking at examples where root finding is useful, we're going to consider a number of root finding methods.

## Bisection method

Here we will use the bisection method to find a root of $f(x) = \sin(\cosh(2 x))$ on the interval $[0, 1]$.

In [ ]:
func = lambda x: np.sin(np.cosh(2 * x))

Let's first look at $f$ on the interval.

In [ ]:
# Set up figure
_, ax = plt.subplots()

# Plot function
xs = np.linspace(0, 1, 500)

plt.plot(xs, func(xs))

plt.grid(alpha=0.4)

ax.set_xlabel(r"$x$");

It looks like $f$ has one root on this interval at $x \approx 0.9$.

Let's code our own version of the bisection algorithm.

In [ ]:
def bisection_method(
    start: float, end: float, f: Callable[[float], float], tol: float = 1e-14
) -> float:
    """Find a root of an interval using the bisection method.
    
    Note that this can be made a bit more efficient, but there's
    no real need since there's already an existing SciPy method
    that implements this more efficiently.
    """
    # Evaluate endpoints
    f_start = f(start)
    f_end = f(end)

    # Check if either of the endpoints are zeros
    if f_start == 0:
        return start

    if f_end == 0:
        return end

    # If difference of endpoints is within tolerance,
    # return the start val
    if end - start < tol:
        return start

    # Get middle point and evaluate it
    middle = (start + end) / 2
    f_middle = f(middle)

    # Check if the middle is zero
    if f_middle == 0:
        return middle

    # Now bisect
    if f_start * f_middle < 0:
        return bisection_method(start, middle, f, tol)

    return bisection_method(middle, end, f, tol)

Let's try using the above function to find the root of $f$.

In [ ]:
x = bisection_method(0, 1, func)

print("x = %.10f" % x)

This is consistent with our guess. Now let's try finding the root using SciPy's version of the bisection algorithm.

In [ ]:
_, r = optimize.bisect(a=0, b=1, f=func, full_output=True)

print(r)

## Newton-Raphson method

For the Newton-Raphson (NR) method, instead of passing in a range in which to find a root, we pass in an initial guess. Going back to the plot of $f$ above, anything near $x = 0.5$ would be a bad initial guess, where the derivative of $f$ is either small or zero.

We also need to pass in the derivative to the NR method. Using the chain rule, we have

\begin{equation}
    \frac{df}{dx} = 2 \sinh (2 x) \cos( \cosh(2 x))
    .
\end{equation}

Let's use the NR method to find the root of $f$, giving it our initial guess of $x = 0.9$.

In [ ]:
_, r = optimize.newton(
    func=func,
    fprime=lambda x: 2 * np.sinh(2 * x) * np.cos(np.cosh(2 * x)),
    x0=0.9,
    full_output=True,
)

print(r)

As we can see by the number of iterations taken, NH is much faster than bisection! 

## Secant method

For the secant method, we essentially perform the NR method, but instead of explicitly using the derivative, we use a linear approximation of it. Let's see how well it performs.

In [ ]:
_, r = optimize.newton(func=func, x0=0.9, full_output=True,)

print(r)

For our function, it only required one more iteration, but also used one fewer function call.

## Brent's method

Now we'll try using Brent's method.

In [ ]:
_, r = optimize.brentq(a=0, b=1, f=func, full_output=True)

print(r)

This is much faster than the bisection method, but still slower than the NR or secant method. However, this has the advantage in that we don't have to provide an initial guess.

## General purpose root finding routine

Now we'll use SciPy's general purpose routine. We need to provide an initial guess for this, so we'll use the same one we used before.

In [ ]:
sol = optimize.root(fun=func, x0=0.9)

print(sol)

Unfortunately I don't know how to interpret the output of this function, so I can't write about how well it performed.

# Root finding applications

Now we'll look at several applications of root finding.

## Lagrange points

For the setup for this problem, see the lab script. Essentially what we need to do is find $r$ such that the following equation holds:

\begin{equation}
    \frac{G M}{r^2} - \frac{G m}{(R - r)^2} = \omega^2 r
    ,
\end{equation}

where $G$ is the gravitational constant; $M$ and $m$ are the masses of the Earth and Moon, respectively; $R$ is the  distance between the Earth and Moon; and $\omega$ is the angular frequency of the satellite.

In [ ]:
M = 5.974e24  # kg
m = 7.348e22  # kg
R = 3.844e8  # m
omega = 2.662e-6  # 1 / s

r = optimize.brentq(
    a=1,
    b=R - 1,
    f=lambda r: constants.G * M / r ** 2
    - constants.G * m / (R - r) ** 2
    - omega ** 2 * r,
)

print("r = %e" % r)

In terms of $R$, we have

In [ ]:
print("r = %.2fR" % (r / R))

## Particle in a finite square well

For a particle of mass $m$ in a finite potential well with "walls" of height $V$ and a length $L$, the allowed energies $E$ ($0 < E < V$) are given by

\begin{align}
    \tan \left( \sqrt{\frac{L^2 m E}{2 \hslash} } \right)
        &= \sqrt{ \frac{V - E}{E} } \qquad \text{(for even numbered states)}, \\
    \tan \left( \sqrt{\frac{L^2 m E}{2 \hslash} } \right)
        &= - \sqrt{ \frac{E}{V - E} } \qquad \text{(for odd numbered states)},
\end{align}

where $\hslash$ is the reduced Planck constant.

Let's plot the left and right hand sides of the equations.

In [ ]:
L = 1e-9  # m
V = 5  # eV

In [ ]:
# Energy range (in eV)
start_e_val = -6
end_e_val = 6

# Get function vals (expect errors here)
e_vals = np.linspace(start_e_val, end_e_val)
lhs_vals = np.tan(
    np.sqrt(
        L ** 2
        * constants.m_e
        * e_vals
        / (2 * constants.physical_constants["reduced Planck constant in eV s"][0])
    )
)
rhs_even_vals = np.sqrt((V - e_vals) / e_vals)
rhs_odd_vals = -np.sqrt(e_vals / (V - e_vals))

In [ ]:
# Set up figure
_, ax = plt.subplots()

# Plot
plt.scatter(e_vals, lhs_vals)
plt.plot(e_vals, rhs_even_vals)
plt.plot(e_vals, rhs_odd_vals);

**FIGURE THIS OUT AND FINISH IT**

# Extremization

Here we'll use a few different minimization methods to find the global minimum of the polynomial $f(x) = 2.1 x^2 - 3.4 x + 2.6$.

In [ ]:
func = lambda x: 2.1 * x ** 2 - 3.4 * x + 2.6

Before finding the minimum of $f$, let's plot it on an interval where it achieves its minimum.

In [ ]:
# Set up figure
_, ax = plt.subplots()

# Plot function
xs = np.linspace(-4, 4, 500)

plt.plot(xs, func(xs))

plt.grid(alpha=0.4)

ax.set_xlabel(r"$x$");

## Brent method

First we'll use the "Brent" method.

In [ ]:
res = optimize.minimize_scalar(fun=func, method="Brent")

print(res)

## Golden search method

Next we'll use the "golden search" method.

In [ ]:
res = optimize.minimize_scalar(fun=func, method="Golden")

print(res)

We can see here that the Brent method uses far less iterations and function evaluations than the golden search method.

## SciPy's minimize

Next we'll use SciPy's `minimize` function. This function requires us to pass in an initial guess; looking at the above plot $x = 1$ seems like a reasonable guess.

In [ ]:
res = optimize.minimize(fun=func, x0=1)

print(res)

This only took one iteration!

# Extremization applications

## Landau free energy of phase transitions

The description of this problem is long and somewhat complicated, so see the lab script for full details. The sort version is that we're going to be considering the Landau free energy for a ferromagnet in terms of its bulk magnetization (order parameter), $M$. The free energy $F$ is given by

\begin{equation}
    F(M) = a M^2 + b M^4 + H M
    ,
\end{equation}

where $b$ is a positive constant; $H$ is the external magnetic field; and $a = a_0 (T - T_c)$, where $T$ is the temperature, $T_c$ is the critical temperature, and $a_0$ is another constant.

First we'll rescale the free energy $F$ to a scaled form $\tilde{f}$ which will be in terms of $t = \frac{T}{T_c}$, $\tilde{b} = \frac{b}{a_0 T_c}$, and $\tilde{h} = \frac{H}{a_0 T_C}$:

\begin{align}
    &F(M) = a M^2 + b M^4 + H M \\
    &\iff F(M) = a_0 (T - T_c) M^2 + b M^4 + H M \\
    &\iff \tilde{f}(M) = (t - 1) M^2 + \tilde{b} M^4 + \tilde{h} M
    .
\end{align}

Let's plot $\tilde{f}$, at $t = 0.4, 0.8. 1.0, 1.4$ with

**Instructions unclear here**

## Ode to incandescent light

The description for this problem is also long, so again see the lab script for the full details.

The power radiated $I$ by an object at temperature $T$ and wavelength $\lambda$ is given by Plank's law:

\begin{equation}
    I(\lambda, T) = 2 \pi A h c^2 \frac{\lambda^5}{\exp \left( \frac{hc}{\lambda k_B T} \right) - 1}
    ,
\end{equation}

where $A$ is the area of the object, $h$ is Plank's constant, $c$ is the speed of light, and $k_B$ is Boltzmann's constant. Note that visible light corresponds to wavelengths between $\lambda_1 = 390 \text{ nm}$ and $\lambda_2 = 750 \text{ nm}$. Thus the total power radiated across the spectrum at temperature $T$ is given by

\begin{equation}
    \int_{\lambda_1}^{\lambda_2} I(\lambda, T) d \lambda
    ,
\end{equation}

while the total power radiated is given by

\begin{equation}
    \int_{0}^{\infty} I(\lambda, T) d \lambda
    .
\end{equation}

Dividing the first integral by the second, we can determine the efficiency of visible light emitted $\eta(T)$. It turns out that we can express $\eta$ by

\begin{equation}
    \eta(T) = \frac{15}{\pi^4} \int_{\frac{hc}{\lambda_2 k_B T}}^{\frac{hc}{\lambda_1 k_B T}}
        \frac{x^3}{e^x - 1}
        d x
    .
\end{equation}

First, let's write a function that calculates $\eta(T)$.

In [ ]:
def efficiency_of_light(T: float) -> float:
    """Calculate \eta(T)."""
    # Endpoints of visible light
    lambda_1 = 390e-9  # m
    lambda_2 = 750e-9  # m

    return (
        15
        / np.pi ** 4
        * integrate.quad(
            func=lambda x: x ** 3 / (np.exp(x) - 1),
            a=constants.h * constants.c / (lambda_2 * constants.k * T),
            b=constants.h * constants.c / (lambda_1 * constants.k * T),
        )[0]
    )

Now let's plot the function for $\eta$ we just wrote.

In [ ]:
# Set up figure
_, ax = plt.subplots()

# Plot function
ts = np.linspace(300, 10 ** 4)

plt.plot(ts, [efficiency_of_light(t) for t in ts])

plt.xscale("log")

ax.set_xlabel(r"$T$")
ax.set_ylabel(r"$\eta$");

It appears there is maximum efficiency at $T \approx 6500 \text{ K}$.

Let's find out what the temperature that leads to maximum efficiency precisely is (as well as what this maximum efficiency is).

**FINISH ME**